In [1]:
import torch
import torch.nn as tnn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn.functional as F

import queue
import os
import numpy as np
from sklearn.manifold import Isomap

In [2]:
os.system('nvidia-smi -q -d Memory |grep -A4 GPU|grep Free >tmp')
memory_gpu=[int(x.split()[2]) for x in open('tmp','r').readlines()]
os.environ['CUDA_VISIBLE_DEVICES']=str(np.argmax(memory_gpu))
os.system('rm tmp')

0

In [3]:
BATCH_SIZE = 50
LEARNING_RATE = 0.01
EPOCH = 1

In [4]:
transform = transforms.Compose([
    transforms.RandomResizedCrop(28),
    transforms.ToTensor()])

data_train = dsets.MNIST(root = "./data/",
                         transform=transform,
                            train = True,
                            download = True)

data_test = dsets.MNIST(root="./data/",
                        transform=transform,
                           train = False)

trainLoader = torch.utils.data.DataLoader(dataset=data_train, batch_size=BATCH_SIZE, shuffle=True)
testLoader = torch.utils.data.DataLoader(dataset=data_test, batch_size=BATCH_SIZE, shuffle=False)

In [5]:
class VGG_conv(tnn.Module):
    def __init__(self):
        super(VGG_conv, self).__init__()
        self.layer1 = tnn.Sequential(

            # 1-1 conv layer
            tnn.Conv2d(1, 64, kernel_size=3, padding=1),
            tnn.BatchNorm2d(64),
            tnn.ReLU(),

            # 1-2 conv layer
            tnn.Conv2d(64, 64, kernel_size=3, padding=1),
            tnn.BatchNorm2d(64),
            tnn.ReLU(),

            # 1 Pooling layer
            tnn.MaxPool2d(kernel_size=2, stride=2))

        self.layer2 = tnn.Sequential(

            # 2-1 conv layer
            tnn.Conv2d(64, 128, kernel_size=3, padding=1),
            tnn.BatchNorm2d(128),
            tnn.ReLU(),

            # 2-2 conv layer
            tnn.Conv2d(128, 128, kernel_size=3, padding=1),
            tnn.BatchNorm2d(128),
            tnn.ReLU(),

            # 2 Pooling lyaer
            tnn.MaxPool2d(kernel_size=2, stride=2))

        self.layer3 = tnn.Sequential(

            # 3-1 conv layer
            tnn.Conv2d(128, 256, kernel_size=3, padding=1),
            tnn.BatchNorm2d(256),
            tnn.ReLU(),

            # 3-2 conv layer
            tnn.Conv2d(256, 256, kernel_size=3, padding=1),
            tnn.BatchNorm2d(256),
            tnn.ReLU(),

            # 3 Pooling layer
            tnn.MaxPool2d(kernel_size=2, stride=2))

        self.layer4 = tnn.Sequential(

            # 4-1 conv layer
            tnn.Conv2d(256, 512, kernel_size=3, padding=1),
            tnn.BatchNorm2d(512),
            tnn.ReLU(),

            # 4-2 conv layer
            tnn.Conv2d(512, 512, kernel_size=3, padding=1),
            tnn.BatchNorm2d(512),
            tnn.ReLU(),

            # 4 Pooling layer
            tnn.MaxPool2d(kernel_size=2, stride=2))

        # self.layer5 = tnn.Sequential(
        #
        #     # 5-1 conv layer
        #     tnn.Conv2d(512, 512, kernel_size=3, padding=1),
        #     tnn.BatchNorm2d(512),
        #     tnn.ReLU(),
        #
        #     # 5-2 conv layer
        #     tnn.Conv2d(512, 512, kernel_size=3, padding=1),
        #     tnn.BatchNorm2d(512),
        #     tnn.ReLU(),
        #
        #     # 5 Pooling layer
        #    tnn.MaxPool2d(kernel_size=2, stride=2))

        self.layer6 = tnn.Sequential(

            # 6 Fully connected layer
            # Dropout layer omitted since batch normalization is used.
            tnn.Linear(512, 512),
            tnn.BatchNorm1d(512),
            tnn.ReLU())


        self.layer7 = tnn.Sequential(

            # 7 Fully connected layer
            # Dropout layer omitted since batch normalization is used.
            tnn.Linear(512, 512,
            tnn.BatchNorm1d(512)),
            tnn.ReLU())
    
    def forward(self, x):
      out = self.layer1(x)
      out = self.layer2(out)
      out = self.layer3(out)
      out = self.layer4(out)
   #   out = self.layer5(out)
      vgg16_features = out.view(out.size(0), -1)
      out = self.layer6(vgg16_features)
      out = self.layer7(out)
      return out

In [33]:
class VGG_fc(tnn.Module):
    def __init__(self):
        super(VGG_fc, self).__init__()
        self.layer8 = tnn.Sequential(

        # 8 output layer
        tnn.Linear(32, 10))

    def forward(self, x):
        out = self.layer8(x)
        out = F.softmax(out, dim=1)
        return out

In [17]:
def isomap(feature_queue, n_components):
    length = feature_queue.qsize()
    for i in range(length):
        if i == 0:
            feature_tmp = feature_queue.get()
            feature_queue.put(feature_tmp)
            features = feature_tmp
        else:
            feature_tmp = feature_queue.get()
            feature_queue.put(feature_tmp)
            features = torch.cat((features, feature_tmp), dim=0)
        
    feature_input = features.detach().cpu().numpy()
    embedding = Isomap(n_components=n_components)
    transformed = embedding.fit_transform(feature_input)
    if features.is_cuda:
        output = torch.from_numpy(transformed).cuda()
    else:
        output = torch.from_numpy(transformed)
    return output
    

In [8]:
feature = torch.ones(1000, 512).cuda()

In [9]:
feature

tensor([[1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        ...,
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.]], device='cuda:0')

In [10]:
feature.cpu().numpy()

array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]], dtype=float32)

In [11]:
isomap(feature, 32)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0', dtype=torch.float64)

In [12]:
vgg_conv = VGG_conv()
# vgg_conv.cuda()
vgg_fc = VGG_fc()
# vgg_fc.cuda()

cost1 = tnn.CosineEmbeddingLoss()
cost2 = tnn.CrossEntropyLoss()
optimizer1 = torch.optim.Adam(vgg_conv.parameters(), lr=LEARNING_RATE)
optimizer2 = torch.optim.Adam(vgg_fc.parameters(), lr=LEARNING_RATE)

In [34]:
# Train the model
for epoch in range(EPOCH):
#  for i, (images, labels) in enumerate(trainLoader):
  vgg_conv.train()
  correct = 0
  total = 0
  train_img_queue = queue.Queue(maxsize=1000/BATCH_SIZE)    #构建输入图像的队列
  train_label_queue = queue.Queue(maxsize=1000/BATCH_SIZE) #构建label的队列
  train_vec_queue = queue.Queue(maxsize=1000/BATCH_SIZE)    #构建卷积网络输出向量的队列
  for images, labels in trainLoader:
    train_img_queue.put(images)
    train_label_queue.put(labels)
    
    # Forward + Backward + Optimize
    
#     optimizer1.zero_grad()
#     optimizer2.zero_grad()

    outputs1 = vgg_conv(images) #卷积网络的输出，将图片embedding成512维向量，the shape of output is (batch_size, 512)
    
    train_vec_queue.put(outputs1)
    
#     print(train_vec_queue.qsize())
#     print(train_vec_queue.get_nowait().shape)
    
    if train_img_queue.full():  #等队列满了之后，开始让所有图片进入isomap，然后pop出队首的数据进行反向传播
        isomap_forward = isomap(train_vec_queue, n_components=32)#将1000张图片通过卷积层得到的embedding向量输入isomap层，获得降维后的结果
        batch_feature = isomap_forward[:BATCH_SIZE].float()
        
        img_tmp = train_img_queue.get()
        label_tmp = train_label_queue.get()
        vec_tmp = train_vec_queue.get()
        
        print(isomap_forward.shape)
        print(batch_feature.shape)
        
        outputs = vgg_fc(batch_feature)
        
        print(outputs.shape)
        
        
        
#         current_img = train_img_queue.get()
#         current_label = train_label_queue.get()
#         current_vec = train_vec_queue.get()
        
    
    
#     _, predicted = torch.max(outputs.data, 1)
#     total += labels.size(0)
#     correct += (predicted.cpu() == labels.cpu()).sum()
#     loss = cost(outputs, labels.cuda())
#     loss.backward()
#     optimizer.step()

#   print ('Epoch [%d/%d], Loss. %.4f' %
#              (epoch+1, EPOCH, loss.data[0]))
#   print('Test Accuracy of the model on the training set: %d %%' % (100 * correct / total))

# # Test the model
#   vgg16.eval()
#   correct = 0
#   total = 0

#   for images, labels in testLoader:
#     images = Variable(images).cuda()
#     outputs = vgg16(images)
#     _, predicted = torch.max(outputs.data, 1)
#     total += labels.size(0)
#     correct += (predicted.cpu() == labels).sum()

#   print('Test Accuracy of the model on the 10000 test images: %d %%' % (100 * correct / total))

# # Save the Trained Model
# torch.save(vgg16.state_dict(),'checkpoint_without_model.pt')


torch.Size([1000, 32])
torch.Size([50, 32])
torch.Size([50, 10])


/home/vrserver/anaconda3/lib/python3.6/site-packages/torch/nn/modules/container.py:92: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


torch.Size([1000, 32])
torch.Size([50, 32])
torch.Size([50, 10])


/home/vrserver/anaconda3/lib/python3.6/site-packages/torch/nn/modules/container.py:92: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


torch.Size([1000, 32])
torch.Size([50, 32])
torch.Size([50, 10])


...

torch.Size([1000, 32])
torch.Size([50, 32])
torch.Size([50, 10])


/home/vrserver/anaconda3/lib/python3.6/site-packages/torch/nn/modules/container.py:92: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


torch.Size([1000, 32])
torch.Size([50, 32])
torch.Size([50, 10])


/home/vrserver/anaconda3/lib/python3.6/site-packages/torch/nn/modules/container.py:92: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


KeyboardInterrupt: 